In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time


In [15]:

def setup_driver():
    chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def wait_for_page_load():
    time.sleep(3)  # Waktu tunggu yang lebih efisien untuk memuat halaman

def scroll_to_bottom(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    wait_for_page_load()


In [16]:
def search_tokopedia(keyword):
    driver = setup_driver()
    product_list = []
    try:
        # Akses ke Tokopedia dan cari produk
        driver.get("https://www.tokopedia.com/")
        wait_for_page_load()

        search_input = driver.find_element(By.CSS_SELECTOR, "input[aria-label='Cari di Tokopedia']")
        search_input.send_keys(keyword, Keys.ENTER)
        wait_for_page_load()

        # Scroll untuk memuat konten tambahan
        scroll_to_bottom(driver)

        # Ambil informasi produk dari 7 halaman pertama
        for page in range(1, 8):
            product_list.extend(extract_product_info(driver))
            navigate_to_next_page(driver, page)

        # Lanjut ke halaman berikutnya jika ada
        while navigate_to_next_page(driver):
            product_list.extend(extract_product_info(driver))

    finally:
        driver.quit()
        # Konversi data produk ke DataFrame dan simpan sebagai CSV
        if product_list:
            df = pd.DataFrame(product_list, columns=['Nama Produk', 'Harga', 'Link'])
            df.to_csv('tokopedia_products.csv', index=False)
            print("Data produk berhasil disimpan ke tokopedia_products.csv")

In [17]:
def extract_product_info(driver):
    product_data = []
    products = driver.find_elements(By.CSS_SELECTOR, ".css-jza1fo .css-5wh65g")
    for product in products:
        try:
            product_name = product.find_element(By.CSS_SELECTOR, "span.OWkG6oHwAppMn1hIBsC3pQ").text
            product_price = product.find_element(By.CSS_SELECTOR, "div._8cR53N0JqdRc+mQCckhS0g").text
            product_link = product.find_element(By.TAG_NAME, "a").get_attribute("href")
            product_data.append({
                'Nama Produk': product_name,
                'Harga': product_price,
                'Link': product_link
            })
        except Exception as e:
            print(f"Gagal mengambil informasi produk: {e}")
    return product_data


In [18]:
def navigate_to_next_page(driver, current_page=None):
    try:
        if current_page:
            pagination_button = driver.find_element(By.XPATH, f"//button[@aria-label='Laman {current_page + 1}']")
            pagination_button.click()
        else:
            next_button = driver.find_element(By.XPATH, "//button[@aria-label='Laman berikutnya']")
            next_button.click()
        wait_for_page_load()
        scroll_to_bottom(driver)
        return True
    except Exception as e:
        print(f"Gagal navigasi ke halaman berikutnya: {e}")
        return False


: 

In [ ]:

def main():
    search_tokopedia("asus")

if __name__ == "__main__":
    main()


Gagal mengambil informasi produk: Message: no such element: Unable to locate element: {"method":"css selector","selector":"span.OWkG6oHwAppMn1hIBsC3pQ"}
  (Session info: chrome=130.0.6723.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7B02C3AB5+28005]
	(No symbol) [0x00007FF7B02283B0]
	(No symbol) [0x00007FF7B00C580A]
	(No symbol) [0x00007FF7B0115A3E]
	(No symbol) [0x00007FF7B0115D2C]
	(No symbol) [0x00007FF7B010937C]
	(No symbol) [0x00007FF7B013BA7F]
	(No symbol) [0x00007FF7B0109246]
	(No symbol) [0x00007FF7B013BC50]
	(No symbol) [0x00007FF7B015B8B3]
	(No symbol) [0x00007FF7B013B7E3]
	(No symbol) [0x00007FF7B01075C8]
	(No symbol) [0x00007FF7B0108731]
	GetHandleVerifier [0x00007FF7B05B643D+3118829]
	GetHandleVerifier [0x00007FF7B0606C90+3448640]
	GetHandleVerifier [0x00007FF7B05FCF0D+3408317]
	GetHandleVerifier [0x00007FF7B038A40B+841403]
	(No 